# PubPulse laboratory

First, a little setup to use the database (output hidden, because it's noisy):

In [ ]:
%%capture
!pip install psycopg2-binary
!pip install ipython-sql
!pip install -U sentence-transformers
!pip install SQLAlchemy --quiet
!pip install pgvector
!pip install tqdm

In [2]:
%load_ext sql
%sql $DATABASE_URL

In [3]:
import sys
sys.path.append('/app')

How many statuses have we ingested so far?

In [19]:
%%sql
SELECT count(*)
FROM statuses

 * postgresql://postgres:***@db:5432/example
1 rows affected.


count
187778


Let's take a look at the latest posts ingested:

In [25]:
%%sql
SELECT
    url,
    ingested_at,
    status->>'created_at' as created_at,
    status->'account'->>'acct' as acct
FROM statuses
ORDER BY ingested_at DESC
LIMIT 5;

 * postgresql://postgres:***@db:5432/example
5 rows affected.


url,ingested_at,created_at,acct
https://mstdn.social/@hkrn/112153459094121265,2024-03-25 00:19:34.353786+00:00,2024-03-25T00:19:34+00:00,hkrn@mstdn.social
https://mstdn.dk/@Hovedorganet/112153459067270187,2024-03-25 00:19:34.184111+00:00,2024-03-25T00:19:34+00:00,Hovedorganet@mstdn.dk
https://social.edist.ro/@nws_oun/112153459003844977,2024-03-25 00:19:32.668426+00:00,2024-03-25T00:19:33+00:00,nws_oun@social.edist.ro
https://mstdn.social/@hkrn/112153458720146348,2024-03-25 00:19:31.414036+00:00,2024-03-25T00:19:29+00:00,hkrn@mstdn.social
https://twitter.com/MenInBlazers/status/1772055141614596394,2024-03-25 00:19:29.830966+00:00,2024-03-25T00:17:23+00:00,MenInBlazers@sportsbots.xyz


Try fetching the latest posts using python:

In [26]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
import pandas as pd 

engine = create_engine(os.environ["DATABASE_URL"])
sql_df = pd.read_sql( 
    """
        SELECT
            url,
            ingested_at,
            status->>'created_at' as created_at,
            status->'account'->>'acct' as acct
        FROM statuses
        ORDER BY ingested_at DESC
        LIMIT 5;
    """, 
    con=engine 
) 
sql_df

,url,ingested_at,created_at,acct
0,https://nya.one/notes/9r9rm33zeuix37ud,2024-03-25 00:19:35.399339+00:00,2024-03-25T00:19:35.615000+00:00,Samukawa_Shion@nya.one
1,https://mstdn.social/@hkrn/112153459094121265,2024-03-25 00:19:34.353786+00:00,2024-03-25T00:19:34+00:00,hkrn@mstdn.social
2,https://mstdn.dk/@Hovedorganet/112153459067270187,2024-03-25 00:19:34.184111+00:00,2024-03-25T00:19:34+00:00,Hovedorganet@mstdn.dk
3,https://social.edist.ro/@nws_oun/1121534590038...,2024-03-25 00:19:32.668426+00:00,2024-03-25T00:19:33+00:00,nws_oun@social.edist.ro
4,https://mstdn.social/@hkrn/112153458720146348,2024-03-25 00:19:31.414036+00:00,2024-03-25T00:19:29+00:00,hkrn@mstdn.social


In [7]:
from sqlalchemy.sql import text
from sqlalchemy import create_engine

engine = create_engine(os.environ["DATABASE_URL"])

with engine.connect() as conn:
    stmt = text("""
        SELECT
            ingested_at,
            url,
            status->>'created_at' as created_at,
            status->'account'->>'acct' as acct,
            status->>'content' as content
        FROM statuses
        ORDER BY ingested_at DESC
        LIMIT 500;
    """)
    result = conn.execute(stmt)
    
from collections import namedtuple

Record = namedtuple('Record', result.keys())
records = [Record(*r) for r in result.fetchall()]

texts = [r.content for r in records if r.content]

len(texts)

483

In [8]:
"""
model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = os.environ["HF_TOKEN"]

import requests

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

def query(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
    return response.json()

output = query(texts)

import pandas as pd
embeddings = pd.DataFrame(output)
embeddings
"""

'\nmodel_id = "sentence-transformers/all-MiniLM-L6-v2"\nhf_token = os.environ["HF_TOKEN"]\n\nimport requests\n\napi_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"\nheaders = {"Authorization": f"Bearer {hf_token}"}\n\ndef query(texts):\n    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})\n    return response.json()\n\noutput = query(texts)\n\nimport pandas as pd\nembeddings = pd.DataFrame(output)\nembeddings\n'

In [30]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [31]:
import os
import psycopg2

conn = psycopg2.connect(os.environ["DATABASE_URL"])

cur = conn.cursor()
cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
cur.execute("DROP TABLE IF EXISTS embeddings")
cur.execute("""
CREATE TABLE IF NOT EXISTS embeddings(
    id INTEGER,
    url character varying NOT NULL UNIQUE,
    embedding vector(384)
)
""")

conn.commit()

In [28]:
%%sql
DELETE
FROM embeddings
WHERE 1=1

 * postgresql://postgres:***@db:5432/example
0 rows affected.


[]

Try generating and storing embedding vectors for the latest bunch of statuses:

In [62]:
from ipywidgets import IntProgress
from IPython.display import display
import time
import os
import numpy as np
import pandas as pd
import psycopg2
from pgvector.psycopg2 import register_vector
from tqdm.notebook import trange, tqdm

conn = psycopg2.connect(os.environ["DATABASE_URL"])
register_vector(conn)

cur = conn.cursor()
cur.execute("""
    SELECT
        url,
        status->>'content' as content
    FROM statuses
    ORDER BY ingested_at DESC
    LIMIT 50000
""")

CHUNK_SIZE = 100
chunks = []

for row in tqdm(cur, total=cur.rowcount):
    chunks.append((row[0], row[1]))

    if len(chunks) >= CHUNK_SIZE:    
        urls = [c[0] for c in chunks]
        embeddings = model.encode([c[1] for c in chunks])
        chunks = []

        with conn:
            with conn.cursor() as cur:
                for idx in range(0, len(urls)):
                    url = urls[idx]
                    embedding = embeddings[idx]
                    cur.execute(
                        """
                            INSERT INTO embeddings (url, embedding) VALUES (%s, %s)
                              ON CONFLICT (url) DO UPDATE SET embedding = EXCLUDED.embedding;            
                        """,
                        (url, embedding)
                    )


  0%|          | 0/50000 [00:00<?, ?it/s]

In [54]:
%%sql
SELECT count(embedding)
FROM embeddings

 * postgresql://postgres:***@db:5432/example
1 rows affected.


count
28959


In [61]:
embeddings = model.encode([
    """blogs are the future of the internet"""
])


conn = psycopg2.connect(os.environ["DATABASE_URL"])
register_vector(conn)

cur = conn.cursor()
cur.execute(
    """
    SELECT
        url, status->>'content'
    FROM statuses
    WHERE url in (
        SELECT url
        FROM embeddings
        ORDER BY embedding <-> %s
        LIMIT 10
    )
    """,
    (np.array(embeddings[0]),)
)
rows = cur.fetchall()

import pandas as pd
pd.DataFrame(rows, columns=("url", "content"))

,url,content
0,https://d20.social/@shane/112152933011129501,<p>I discovered Max’s site after looking aroun...
1,https://mastodonapp.uk/@requiem/11215357139011...,"<p>IDEA: Blogging to <a href=""https://mastodon..."
2,https://wetdry.world/@Nikki_Asdo/1121533828179...,<p>i should talk about my interests instead of...
3,https://mastodon.social/@alturasolutions/11215...,<p>How to Keep Blogging Real</p><p>We live in ...
4,https://mastodon.world/@paninid/11215334230396...,<p>Yahoo News developing a platform for creato...
5,https://podcastindex.social/@js/11215341774906...,<p>‘Feedle - A search engine for blogs and pod...
6,https://mastodon.social/@levelbot/112153103830...,<p>blogging levels are currently at 35% but fl...
7,https://mastodon.social/@Lafargue/112153017674...,<p>Got a lot of &#39;moral philosophy&#39; / p...
8,https://social.masto.land/@dave/11215369727290...,"<p>Here is the MetaWeblog API, the common glue..."
9,https://wetdry.world/@iagondiscord/11215331079...,<p>also I can't wait to get my own VPS so I ca...
